Visto che i passi mi serviranno praticamente sempre in forma normalizzata, mi salvo anche la versione del dataset con la colonna dei passi normalizzati.
Inoltre probabilmente è meglio avere anche lo score in forma normalizzata.
Normalizzo anche le date usando minmax.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/valutati.csv.gz')

In [3]:
from sklearn.preprocessing import MinMaxScaler
import datetime

df["date"]=pd.to_datetime(df["date"],format= "%Y-%m-%d")
df["n_date"] = MinMaxScaler().fit_transform(df[["date"]])


Faccio le normalizzazioni sia utente per utente che su tutto il dataset

In [4]:
from sklearn.preprocessing import StandardScaler  # Questo scaler è meno sensibile agli outliers


def scale(x):
    x["n_steps"] = StandardScaler().fit_transform(x[["steps"]])  # Lo scaling è fatto sulla base dell'utente
    x["n_sleep_score"] = StandardScaler().fit_transform(x[["sleep_score"]])  # Lo scaling è fatto sulla base dell'utente
    return x


def scale2(x):
    x["n_steps_all"] = StandardScaler().fit_transform(x[["steps"]])  # Lo scaling è fatto sulla base dell'utente
    x["n_sleep_score_all"] = StandardScaler().fit_transform(
        x[["sleep_score"]])  # Lo scaling è fatto sulla base dell'utente
    return x


# df = df.groupby("uuid").apply(scale, include_groups=False)
df = df.transform(scale2)
df.head()

,uuid,date,steps,sleep_score,n_date,n_steps_all,n_sleep_score_all
0,1003e58667235e01b49008155604980b3900b00e,2016-07-31,3049.04,NaN,0.332418,-0.866270,NaN
1,1003e58667235e01b49008155604980b3900b00e,2016-08-01,4309.81,0.547576,0.335165,-0.600012,-0.367737
2,1003e58667235e01b49008155604980b3900b00e,2016-08-02,255.11,NaN,0.337912,-1.456310,NaN
3,1003e58667235e01b49008155604980b3900b00e,2016-08-03,5118.07,NaN,0.340659,-0.429319,NaN
4,1003e58667235e01b49008155604980b3900b00e,2016-08-04,2228.88,0.415384,0.343407,-1.039477,-1.259176


In [5]:
# df = df.rename_axis(["uuid", "id"]).reset_index().drop(
#     columns=["id"])  # Per un csv più pulito, infatti il groupby aggiunge una colonna che non voglio
# df.head()

In [6]:
def rolling_mean(x):
    y=x[::-1].rolling(window='10D', min_periods=6, on="date")[[
        "steps",
        # "n_steps",
        "n_steps_all",
        "sleep_score",
        # "n_sleep_score",
        "n_sleep_score_all"
    ]].mean()[::-1]
    x["m_steps"] = y["steps"]
    x["dm_steps"] = y["steps"].diff(1)
    # x["mn_steps"]= y["n_steps"]
    # x["dmn_steps"]= y["n_steps"].diff(1)
    x["mn_steps_all"]= y["n_steps_all"]
    x["dmn_steps_all"]= y["n_steps_all"].diff(1)
    x["m_sleep_score"]= y["sleep_score"]
    x["dm_sleep_score"]= y["sleep_score"].diff(1)
    # x["mn_sleep_score"]= y["n_sleep_score"]
    # x["dmn_sleep_score"]= y["n_sleep_score"].diff(1)
    x["mn_sleep_score_all"] = y["n_sleep_score_all"]
    x["dmn_sleep_score_all"] = y["n_sleep_score_all"].diff(1)
    return x

df = df.groupby("uuid").apply(rolling_mean, include_groups=False)
df = df.rename_axis(["uuid", "id"]).reset_index().drop(columns=["id"])  # Per un csv più pulito, infatti il groupby aggiunge una colonna che non voglio
df.head(100)

,uuid,date,steps,sleep_score,n_date,n_steps_all,n_sleep_score_all,m_steps,dm_steps,mn_steps_all,dmn_steps_all,m_sleep_score,dm_sleep_score,mn_sleep_score_all,dmn_sleep_score_all
0,1003e58667235e01b49008155604980b3900b00e,2016-07-31,3049.04,NaN,0.332418,-0.866270,NaN,3243.864,NaN,-0.825126,NaN,NaN,NaN,NaN,NaN
1,1003e58667235e01b49008155604980b3900b00e,2016-08-01,4309.81,0.547576,0.335165,-0.600012,-0.367737,3673.029,429.165000,-0.734492,0.090634,NaN,NaN,NaN,NaN
2,1003e58667235e01b49008155604980b3900b00e,2016-08-02,255.11,NaN,0.337912,-1.456310,NaN,4083.098,410.069000,-0.647891,0.086601,NaN,NaN,NaN,NaN
3,1003e58667235e01b49008155604980b3900b00e,2016-08-03,5118.07,NaN,0.340659,-0.429319,NaN,4295.129,212.031000,-0.603113,0.044778,NaN,NaN,NaN,NaN
4,1003e58667235e01b49008155604980b3900b00e,2016-08-04,2228.88,0.415384,0.343407,-1.039477,-1.259176,4306.411,11.282000,-0.600730,0.002383,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1003e58667235e01b49008155604980b3900b00e,2017-01-07,17983.19,NaN,0.771978,2.287620,NaN,9219.420,359.394286,0.436831,0.075899,NaN,NaN,NaN,NaN
96,1003e58667235e01b49008155604980b3900b00e,2017-01-08,6122.89,NaN,0.774725,-0.217115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,1003e58667235e01b49008155604980b3900b00e,2017-01-09,11630.91,NaN,0.777473,0.946104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,1003e58667235e01b49008155604980b3900b00e,2017-01-10,8695.65,NaN,0.780220,0.326217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.to_csv('../data/scalati.csv.gz', compression='gzip', index=False)